## Start date : July 10th, 2023 

### Data Exploration of 2018, 2019, 2020, 2022 Business Data

Author : Kenthia

Goals:

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('display.max_columns', None)
from shapely.geometry import Point
import fiona
import math
import plotly.express as px
from thefuzz import fuzz
from thefuzz import process

# 2018 DATA

In [61]:
#reading in the dataset 

stores_2018 = pd.read_csv('/srv/data/my_shared_data_folder/rafi/2018_Business_Academic_QCQ_grocery.csv')
stores_2018.drop(['Unnamed: 0'],inplace=True,axis=1)
territories = ['PR', 'FM', 'MP', 'GU', 'VI', 'MH']
stores_2018 = stores_2018.drop(stores_2018[stores_2018['STATE'].isin(territories)].index)
stores_2018.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
1139,F L ROBERTS & CO,6 LEALAND AVE,AGAWAM,MA,1001,NaN,13.0,413,2,A,A,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,554101.0,SERVICE STATIONS-GASOLINE & OIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,2.0,447.0,2,NaN,NaN,NaN,NaN,735565151,NaN,3509882.0,25.0,47050.0,D,G,417389415.0,NaN,6,813209.0,3.0,42.065818,-72.622156,X,44140.0,2.0,521.0,25013.0
1140,SS BROTHERS LLC,39 SOUTHWICK,AGAWAM,MA,1001,NaN,13.0,413,2,A,A,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,21303.0,2.0,447.0,9,NaN,NaN,NaN,NaN,439311777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813209.0,3.0,42.065818,-72.622156,X,44140.0,2.0,521.0,25013.0
1141,SODA & PET FOOD CITY,151 SPRINGFIELD ST,AGAWAM,MA,1001,1553.0,13.0,413,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,519112.0,FEED-DEALERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,2.0,447.0,9,NaN,NaN,NaN,NaN,422566537,NaN,NaN,NaN,NaN,NaN,NaN,422566537.0,NaN,6,813207.0,1.0,42.088733,-72.633365,P,44140.0,2.0,521.0,25013.0
1142,PRIDE STORES,395 MAIN ST,AGAWAM,MA,1001,1849.0,13.0,413,2,A,A,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,21303.0,2.0,447.0,2,NaN,NaN,NaN,NaN,423530241,NaN,453449068.0,NaN,34900.0,NaN,G,NaN,NaN,6,813209.0,1.0,42.079505,-72.614108,P,44140.0,2.0,521.0,25013.0
1143,PAJER SUPERETTE,400 COOPER ST,AGAWAM,MA,1001,2139.0,13.0,413,2,A,B,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,82251.0,3.0,670.0,9,NaN,NaN,NaN,NaN,106190515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813207.0,1.0,42.078401,-72.630473,0,44140.0,2.0,521.0,25013.0


In [62]:
#Cleaned lists from the string matches of supermarkets/companies actually owned by the parent coprporation
#***NOTE***There will be a margin of error from this process because stores were validated through manual checks
#***NOTE***It was unrealistic to manually inspect every store generated this way so there will be supermarkets/companies that do not actually belong to the parent coporation
#***NOTE***STOP AND SHOP SUPERMARKET was missing from string match so it was manually added
#KROGERS FINAL
kroger_clean = ["BAKER'S",
 'BAKERS', 
 'CITY MARKET',
 'DILLONS',
 'FOOD 4 LESS', 
 'FOODS CO', 
 'FRED MEYER', 
 "FRY'S FOOD",
 'GERBES SUPER MARKET',  
 'GERBES SUPER MARKETS', 
 'GERBES SUPERMARKET', 
 'HARRIS TEETER', 
 'JAYC', 
 'KING SOOPERS',
 'KROGER',
 "MARIANO'S",
 'METRO MARKET', 
 'PAY LESS SUPER MARKETS',
 "PICK'N SAVE", 
 'QFC', 
 'RALPHS', 
 'RULER FOODS', 
 "SMITH'S"]

#Ahold Delhaize FINAL:
ahold_clean = [ 'FOOD LION',
 'GIANT',
 'GIANT EAGLE',
 'GIANT FOOD',
 'HANNAFORD',
 'STOP & SHOP',
 'BFRESH MARKET',
 'EASTSIDE MARKETPLACE','STOP & SHOP SUPERMARKET']

#WALMART FINAL:
walmart_clean = ['WALMART GROCERY PICKUP',
 'WALMART GROCERY PKUP-DELIVERY', 
 "SAM'S CLUB DELI"]

#COSTCO FINAL:
costco_clean = ["COSTCO DELI"]

#ALBERTSONS FINAL:
albertsons_clean = ['ACME MARKETS',
'ALBERTSONS',
'ALBERTSONS MARKET',
'AMIGOS UNITED',
"ANDRONICO'S COMMUNITY MARKETS",
'CARRS/SAFEWAY',
'CARRS SAFEWAY',
'HAGGEN',
'HAGGEN FOOD & PHARMACY',
'HAGGEN FOOD',
'JEWEL-OSCO',
'KINGS FOOD MARKETS',
'KINGS FOOD MARKET',
'MARKET STREET',
"PAK 'N SAVE",
'PAVILIONS',
'RANDALLS',
'SAFEWAY',
'STAR MARKET',
'TOM THUMB',
'UNITED SUPERMARKETS',
'VONS',
'SUPER SAVER',
'CARRS QUALITY CTR PALMER SHPG',
 "SAAR'S SUPER SAVER FOODS"]

In [63]:
#placing function here to run storecount below on grocery_naics


def parent_name(row):
    '''Creates a new column called "PARENT NAME" based on the cleaned string match list''' 
    '''The new column generalizes individual companies into their parent coporation (ie. Walmart, Krogers, etc.)''' 
    companies = row['COMPANY']  
    
    if companies in walmart_clean:
        return 'WALMART'
    if companies in costco_clean:
        return 'COSTCO'
    if companies in ahold_clean:
        return 'AHOLD DELHAIZE'
    if companies in kroger_clean:
        return 'KROGER'
    if companies in albertsons_clean:
        return 'ALBERTSONS'
    else:
        return 'OTHER'

stores_2018['PARENT NAME'] = stores_2018.apply(parent_name, axis=1)

In [64]:
#filter by NAICS code "44511" - SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS

grocery_naics_2018 = stores_2018.loc[stores_2018[['PRIMARY NAICS CODE']].astype(str).apply(lambda x: x.str.startswith('44511')).any(axis=1)]
grocery_naics_2018

# There are 75227 supermarkets/related companies in the entire US in 2018

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE,PARENT NAME
1141,SODA & PET FOOD CITY,151 SPRINGFIELD ST,AGAWAM,MA,1001,1553.0,13.0,413,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,519112.0,FEED-DEALERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,2.0,447.0,9,NaN,NaN,NaN,NaN,422566537,NaN,NaN,NaN,NaN,NaN,NaN,422566537.0,NaN,6,813207.0,1.0,42.088733,-72.633365,P,44140.0,2.0,521.0,25013.0,OTHER
1143,PAJER SUPERETTE,400 COOPER ST,AGAWAM,MA,1001,2139.0,13.0,413,2,A,B,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,82251.0,3.0,670.0,9,NaN,NaN,NaN,NaN,106190515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813207.0,1.0,42.078401,-72.630473,0,44140.0,2.0,521.0,25013.0,OTHER
1146,GEISSLER'S SUPERMARKET,830 SUFFIELD ST,AGAWAM,MA,1001,2928.0,13.0,860,2,E,G,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,82251.0,90.0,20079.0,2,L,NaN,NaN,NaN,961890431,NaN,402882462.0,NaN,1092112.0,NaN,K,NaN,NaN,6,813205.0,2.0,42.061261,-72.632281,0,44140.0,2.0,521.0,25013.0,OTHER
1147,CARANDO GOURMET FROZEN FOODS,175 MAIN ST,AGAWAM,MA,1001,1804.0,13.0,413,2,F,G,541104,FOOD PRODUCTS-RETAIL,44511002.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,100.0,22310.0,9,NaN,1933.0,NaN,NaN,417287788,NaN,NaN,NaN,NaN,NaN,NaN,417287788.0,NaN,6,813209.0,1.0,42.086540,-72.620072,P,44140.0,2.0,521.0,25013.0,OTHER
1148,DAIRY STORE,335 SUFFIELD ST,AGAWAM,MA,1001,2133.0,13.0,413,2,A,A,541101,FOOD MARKETS,44511001.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,50107.0,2.0,447.0,9,NaN,NaN,NaN,NaN,302529631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813209.0,3.0,42.077923,-72.629794,0,44140.0,2.0,521.0,25013.0,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162707,BLACK BEAR STORE,6624 BIG SALT RD,KLAWOCK,AK,99925,NaN,198.0,907,2,C,C,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,541103.0,CONVENIENCE STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39106.0,10.0,2283.0,9,NaN,NaN,NaN,NaN,877807065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,200.0,1.0,55.539085,-132.999950,X,0.0,NaN,0.0,2198.0,OTHER
162708,LEASK'S MARKET,1ST & MILTON ST,METLAKATLA,AK,99926,NaN,198.0,907,2,C,D,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39106.0,15.0,3424.0,9,NaN,1952.0,NaN,NaN,877836940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,940100.0,3.0,55.087048,-131.542403,X,0.0,NaN,0.0,2198.0,OTHER
162709,POINT BAKER TRADING POST,PO BOX 130,POINT BAKER,AK,99927,130.0,198.0,907,2,A,B,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39106.0,3.0,685.0,9,NaN,NaN,NaN,NaN,877749747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,100.0,1.0,56.258609,-133.340369,X,0.0,NaN,0.0,2198.0,OTHER
162

In [65]:
grocery_type= grocery_naics_2018["PRIMARY NAICS CODE"].unique().tolist()
grocery_type

[44511003.0,
 44511002.0,
 44511001.0,
 44511004.0,
 44511007.0,
 44511005.0,
 44511006.0,
 44511008.0]

In [66]:
store_count_2018= grocery_naics_2018.groupby('PARENT NAME').count()[["CITY"]].reset_index().rename(columns={"CITY":"STORE COUNT"})
store_count_2018

,PARENT NAME,STORE COUNT
0,AHOLD DELHAIZE,1648
1,ALBERTSONS,2153
2,KROGER,2366
3,OTHER,69060


# 2019 DATA

In [67]:
#reading in the dataset 

stores_2019 = pd.read_csv('/srv/data/my_shared_data_folder/rafi/2019_Business_Academic_QCQ_grocery.csv')
stores_2019.drop(['Unnamed: 0'],inplace=True,axis=1)
territories = ['PR', 'FM', 'MP', 'GU', 'VI', 'MH']
stores_2019 = stores_2019.drop(stores_2019[stores_2019['STATE'].isin(territories)].index)
stores_2019.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
1069,SODA & PET FOOD CITY,151 SPRINGFIELD ST,AGAWAM,MA,1001,1553.0,13.0,413,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,519112.0,FEED-DEALERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,2.0,447.0,9,NaN,NaN,NaN,NaN,422566537,NaN,NaN,NaN,NaN,NaN,NaN,422566537.0,NaN,6,813207.0,1.0,42.088733,-72.633365,P,44140.0,2.0,0.0,25013.0
1070,PAJER SUPERETTE,400 COOPER ST,AGAWAM,MA,1001,2139.0,13.0,413,2,A,B,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,82251.0,3.0,670.0,9,NaN,NaN,NaN,NaN,106190515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813207.0,1.0,42.078751,-72.630074,P,44140.0,2.0,0.0,25013.0
1071,PRIDE STORES,395 MAIN ST,AGAWAM,MA,1001,1849.0,13.0,413,2,A,A,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,21303.0,2.0,447.0,2,NaN,NaN,NaN,NaN,423530241,NaN,453449068.0,NaN,37950.0,NaN,G,NaN,NaN,6,813209.0,1.0,42.079505,-72.614108,P,44140.0,2.0,0.0,25013.0
1072,DAIRY STORE,335 SUFFIELD ST,AGAWAM,MA,1001,2133.0,13.0,413,2,A,A,541101,FOOD MARKETS,44511001.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,50107.0,2.0,447.0,9,NaN,NaN,NaN,NaN,302529631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813209.0,3.0,42.077753,-72.629787,P,44140.0,2.0,0.0,25013.0
1073,SOUTHBRIDGE CONVENIENCE,1 BRIDGE ST,AGAWAM,MA,1001,NaN,13.0,413,2,A,A,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,21303.0,2.0,447.0,9,NaN,NaN,NaN,NaN,423736193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813209.0,3.0,42.065818,-72.622156,X,44140.0,2.0,0.0,25013.0


In [68]:
#placing function here to run storecount below on grocery_naics


def parent_name(row):
    '''Creates a new column called "PARENT NAME" based on the cleaned string match list''' 
    '''The new column generalizes individual companies into their parent coporation (ie. Walmart, Krogers, etc.)''' 
    companies = row['COMPANY']  
    
    if companies in walmart_clean:
        return 'WALMART'
    if companies in costco_clean:
        return 'COSTCO'
    if companies in ahold_clean:
        return 'AHOLD DELHAIZE'
    if companies in kroger_clean:
        return 'KROGER'
    if companies in albertsons_clean:
        return 'ALBERTSONS'
    else:
        return 'OTHER'

stores_2019['PARENT NAME'] = stores_2019.apply(parent_name, axis=1)

In [69]:
#filter by NAICS code "44511" - SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS

grocery_naics_2019 = stores_2019.loc[stores_2019[['PRIMARY NAICS CODE']].astype(str).apply(lambda x: x.str.startswith('44511')).any(axis=1)]
grocery_naics_2019

# There 81480 supermarkets/related companies in the entire US in 2019

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE,PARENT NAME
1069,SODA & PET FOOD CITY,151 SPRINGFIELD ST,AGAWAM,MA,1001,1553.0,13.0,413,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,519112.0,FEED-DEALERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,2.0,447.0,9,NaN,NaN,NaN,NaN,422566537,NaN,NaN,NaN,NaN,NaN,NaN,422566537.0,NaN,6,813207.0,1.0,42.088733,-72.633365,P,44140.0,2.0,0.0,25013.0,OTHER
1070,PAJER SUPERETTE,400 COOPER ST,AGAWAM,MA,1001,2139.0,13.0,413,2,A,B,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,82251.0,3.0,670.0,9,NaN,NaN,NaN,NaN,106190515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813207.0,1.0,42.078751,-72.630074,P,44140.0,2.0,0.0,25013.0,OTHER
1072,DAIRY STORE,335 SUFFIELD ST,AGAWAM,MA,1001,2133.0,13.0,413,2,A,A,541101,FOOD MARKETS,44511001.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,50107.0,2.0,447.0,9,NaN,NaN,NaN,NaN,302529631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,813209.0,3.0,42.077753,-72.629787,P,44140.0,2.0,0.0,25013.0,OTHER
1077,GEISSLER'S SUPERMARKET,830 SUFFIELD ST,AGAWAM,MA,1001,2928.0,13.0,860,2,E,G,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,82251.0,90.0,20079.0,2,L,NaN,NaN,NaN,961890431,NaN,402882462.0,NaN,1169133.0,NaN,K,NaN,NaN,6,813205.0,2.0,42.061261,-72.632281,0,44140.0,2.0,0.0,25013.0,OTHER
1078,CARANDO GOURMET FROZEN FOODS,175 MAIN ST,AGAWAM,MA,1001,1804.0,13.0,413,2,F,G,541104,FOOD PRODUCTS-RETAIL,44511002.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,100.0,22310.0,9,NaN,1933.0,NaN,NaN,417287788,NaN,NaN,NaN,NaN,NaN,NaN,417287788.0,NaN,6,813209.0,1.0,42.086540,-72.620072,P,44140.0,2.0,0.0,25013.0,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177543,LEASK'S MARKET,1ST & MILTON ST,METLAKATLA,AK,99926,NaN,198.0,907,2,C,D,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,39106.0,15.0,3424.0,9,NaN,1952.0,NaN,NaN,877836940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,940100.0,3.0,55.087048,-131.542403,X,0.0,NaN,0.0,2198.0,OTHER
177544,POINT BAKER TRADING POST,PO BOX 130,POINT BAKER,AK,99927,130.0,198.0,907,2,A,B,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,39106.0,3.0,685.0,9,NaN,NaN,NaN,NaN,877749747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,100.0,1.0,56.258606,-133.340366,4,0.0,NaN,0.0,2198.0,OTHER
177545,CITY MARKET WRANGELL,423 FRONT ST,WRANGELL,AK,99929,NaN,275.0,907,2,D,E,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,592102.0,LIQUORS-RETAIL,542107.0,MEAT-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,2019,39106.0,30.0,5217.0,9,NaN,1924.0,NaN,NaN,175837863,NaN,NaN,NaN,NaN,NaN,NaN,175837863.0,NaN,3,300.0,3.0,56.471668,-132.387186,X,0.0,NaN,0.0,2275.

In [70]:
grocery_type= grocery_naics_2019["PRIMARY NAICS CODE"].unique().tolist()
grocery_type

[44511003.0,
 44511001.0,
 44511002.0,
 44511004.0,
 44511007.0,
 44511005.0,
 44511006.0,
 44511008.0]

In [71]:
store_count_2019= grocery_naics_2019.groupby('PARENT NAME').count()[["CITY"]].reset_index().rename(columns={"CITY":"STORE COUNT"})
store_count_2019

,PARENT NAME,STORE COUNT
0,AHOLD DELHAIZE,1759
1,ALBERTSONS,2081
2,KROGER,2269
3,OTHER,75371


# 2020 DATA

In [72]:
#reading in the dataset 

stores_2020 = pd.read_csv('/srv/data/my_shared_data_folder/rafi/2020_Business_Academic_QCQ_grocery.csv')
stores_2020.drop(['Unnamed: 0'],inplace=True,axis=1)
territories = ['PR', 'FM', 'MP', 'GU', 'VI', 'MH']
stores_2020 = stores_2020.drop(stores_2020[stores_2020['STATE'].isin(territories)].index)
stores_2020.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,T&K KWIK STOP,5212 ICELAND AVE,BIRMINGHAM,AL,35224,2405.0,73.0,205.0,2,A,B,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,541105.0,GROCERS-RETAIL,554101.0,SERVICE STATIONS-GASOLINE & OIL,554110.0,ALTERNATIVE FUELS,NaN,NaN,NaN,NaN,2020,21303.0,3.0,511.0,9,NaN,2006.0,NaN,NaN,391201001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,12500.0,1.0,33.521973,-86.930971,P,13820.0,2.0,142.0,1073.0
1,RIVERSIDE GROCERY & DELI,13050 US HIGHWAY 78,RIVERSIDE,AL,35135,1300.0,115.0,205.0,2,A,NaN,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,2.0,NaN,9,NaN,2019.0,NaN,NaN,747150592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,40201.0,1.0,33.600210,-86.198582,0,13820.0,2.0,142.0,1115.0
2,PIGGLY WIGGLY,1619 FORESTDALE BLVD,BIRMINGHAM,AL,35214,2037.0,73.0,205.0,2,D,E,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,39106.0,49.0,8345.0,2,X,NaN,NaN,NaN,620791939,446728776.0,8790206.0,1200.0,20400000.0,I,K,NaN,NaN,6,12401.0,1.0,33.569817,-86.898277,P,13820.0,2.0,142.0,1073.0
3,FORESTADLE JETPEP,1084 FORESTDALE BLVD,BIRMINGHAM,AL,35214,3856.0,73.0,205.0,2,B,C,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,21303.0,6.0,1022.0,9,NaN,NaN,NaN,NaN,803588243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,12402.0,1.0,33.556967,-86.889994,P,13820.0,2.0,142.0,1073.0
4,MURPHY USA,PO BOX 1091,SUMITON,AL,35148,1091.0,127.0,205.0,2,B,C,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,554110.0,ALTERNATIVE FUELS,554101.0,SERVICE STATIONS-GASOLINE & OIL,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,5.0,1796.0,2,M,NaN,NaN,NaN,209415660,NaN,433922353.0,500.0,14034600.0,H,K,NaN,NaN,4,21800.0,2.0,33.752752,-87.043621,4,27530.0,1.0,142.0,1127.0


In [73]:
#placing function here to run storecount below on grocery_naics


def parent_name(row):
    '''Creates a new column called "PARENT NAME" based on the cleaned string match list''' 
    '''The new column generalizes individual companies into their parent coporation (ie. Walmart, Krogers, etc.)''' 
    companies = row['COMPANY']  
    
    if companies in walmart_clean:
        return 'WALMART'
    if companies in costco_clean:
        return 'COSTCO'
    if companies in ahold_clean:
        return 'AHOLD DELHAIZE'
    if companies in kroger_clean:
        return 'KROGER'
    if companies in albertsons_clean:
        return 'ALBERTSONS'
    else:
        return 'OTHER'

stores_2020['PARENT NAME'] = stores_2020.apply(parent_name, axis=1)

In [74]:
#filter by NAICS code "44511" - SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS

grocery_naics_2020 = stores_2020.loc[stores_2020[['PRIMARY NAICS CODE']].astype(str).apply(lambda x: x.str.startswith('44511')).any(axis=1)]
grocery_naics_2020

# There are 76579 supermarkets/related companies in the entire US in 2020

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE,PARENT NAME
1,RIVERSIDE GROCERY & DELI,13050 US HIGHWAY 78,RIVERSIDE,AL,35135,1300.0,115.0,205.0,2,A,NaN,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,2.0,NaN,9,NaN,2019.0,NaN,NaN,747150592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,40201.0,1.0,33.600210,-86.198582,0,13820.0,2.0,142.0,1115.0,OTHER
2,PIGGLY WIGGLY,1619 FORESTDALE BLVD,BIRMINGHAM,AL,35214,2037.0,73.0,205.0,2,D,E,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,39106.0,49.0,8345.0,2,X,NaN,NaN,NaN,620791939,446728776.0,8790206.0,1200.0,20400000.0,I,K,NaN,NaN,6,12401.0,1.0,33.569817,-86.898277,P,13820.0,2.0,142.0,1073.0,OTHER
7,MAIL BOX GROCERIES,12150 INDUSTRY WAY # 3,ANCHORAGE,AK,99515,3595.0,20.0,907.0,2,B,C,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,6.0,1923.0,9,NaN,NaN,NaN,NaN,402363412,NaN,NaN,NaN,NaN,NaN,NaN,425750012.0,NaN,7,2702.0,3.0,61.110638,-149.862679,P,11260.0,2.0,0.0,2020.0,OTHER
8,SWANSONS,830 RIVER ST,BETHEL,AK,99559,NaN,50.0,907.0,2,E,E,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,531102.0,DEPARTMENT STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,50.0,7128.0,9,NaN,2015.0,NaN,NaN,708953051,NaN,NaN,NaN,NaN,NaN,NaN,708953051.0,NaN,5,200.0,4.0,60.837800,-161.617600,X,0.0,NaN,0.0,2050.0,OTHER
11,PORTAGE CREEK GEN STORE-LODGE,1 GUSUK RD,DILLINGHAM,AK,99576,NaN,70.0,907.0,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,701101.0,HOTELS & MOTELS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,2.0,475.0,9,NaN,NaN,NaN,NaN,983555939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,200.0,2.0,59.039600,-158.463000,X,0.0,NaN,0.0,2070.0,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176494,HIGHLANDS ORGANIC MARKET,101 VISTA WAY,KENNEWICK,WA,99336,3120.0,5.0,509.0,2,C,D,541108,GROCERS-HEALTH FOODS,44511007.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,549904.0,VITAMIN & FOOD SUPPLEMENTS,549908.0,FOOD SUPPLEMENTS,549901.0,HEALTH & DIET FOODS-RETAIL,NaN,NaN,NaN,NaN,2020,NaN,15.0,4895.0,9,NaN,NaN,NaN,NaN,175563444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,11100.0,1.0,46.210485,-119.157172,P,28420.0,2.0,313.0,53005.0,OTHER
176498,HAROLD'S GENERAL MERCHANDISE,4080 HARRAH RD,HARRAH,WA,98933,NaN,77.0,509.0,2,B,C,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,39106.0,8.0,1962.0,9,NaN,NaN,NaN,NaN,175393305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,940001.0,1.0,46.405000,-120.577400,X,49420.0,2.0,0.0,53077.0,OTHER
176501,WORLD BISTRO MARKET LLC,PO BOX 871511,VANCOUVER,WA,98687,1511.0,11.0,360.0,2,A,C,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,3.0,1369.0,9,NaN,2018.0,NaN,NaN,737840034,NaN,N

In [75]:
grocery_type= grocery_naics_2020["PRIMARY NAICS CODE"].unique().tolist()
grocery_type

[44511003.0,
 44511007.0,
 44511002.0,
 44511006.0,
 44511004.0,
 44511008.0,
 44511001.0]

In [76]:
store_count_2020= grocery_naics_2020.groupby('PARENT NAME').count()[["CITY"]].reset_index().rename(columns={"CITY":"STORE COUNT"})
store_count_2020

,PARENT NAME,STORE COUNT
0,AHOLD DELHAIZE,1818
1,ALBERTSONS,2090
2,COSTCO,26
3,KROGER,2486
4,OTHER,70149
5,WALMART,10


# 2022 DATA

In [77]:
#reading in the dataset 

stores_2022 = pd.read_csv('/srv/data/my_shared_data_folder/rafi/2022_Business_Academic_QCQ_grocery.csv')
stores_2022.drop(['Unnamed: 0'],inplace=True,axis=1)
territories = ['PR', 'FM', 'MP', 'GU', 'VI', 'MH']
stores_2022 = stores_2022.drop(stores_2022[stores_2022['STATE'].isin(territories)].index)
stores_2022.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,GOMART,55 POSTAL PLZ,MORGANTOWN,WV,26508,7005.0,61.0,304,2,C,C,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,554101.0,SERVICE STATIONS-GASOLINE & OIL,554110.0,ALTERNATIVE FUELS,NaN,NaN,NaN,NaN,NaN,NaN,2022,21303.0,13.0,2482.0,2,NaN,NaN,NaN,NaN,998372387,NaN,124929449.0,50.0,389500.0,E,I,998372387.0,NaN,7,11000.0,3.0,39.594376,-79.954437,P,34060.0,2.0,390.0,54061.0
1,7-ELEVEN,485 E MAIN ST,EL CENTRO,CA,92243,2619.0,25.0,760,2,B,C,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,554101.0,SERVICE STATIONS-GASOLINE & OIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,6.0,1273.0,2,W,NaN,NaN,NaN,495660326,NaN,5863311.0,800.0,1272634.0,H,K,495660326.0,NaN,7,11400.0,3.0,32.792679,-115.536058,0,20940.0,2.0,0.0,6025.0
2,EL SOL MARKET,110 W MAIN ST,WESTMORLAND,CA,92281,NaN,25.0,760,2,B,C,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,39106.0,5.0,1061.0,9,NaN,NaN,NaN,NaN,519605455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,10200.0,1.0,33.051800,-115.581800,X,20940.0,2.0,0.0,6025.0
3,COOL SPRINGS GROCERY,241 WILDWOOD ST,MORGANTOWN,WV,26505,3141.0,61.0,304,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,39106.0,2.0,382.0,9,NaN,NaN,NaN,NaN,817953730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,10400.0,4.0,39.652885,-79.986398,P,34060.0,2.0,390.0,54061.0
4,CIRCLE K,123 E MAIN ST,WESTMORLAND,CA,92281,NaN,25.0,760,2,B,C,541103,CONVENIENCE STORES,44512001.0,CONVENIENCE STORES,554101.0,SERVICE STATIONS-GASOLINE & OIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,6.0,1273.0,2,F,NaN,NaN,NaN,855113817,NaN,450720289.0,650.0,4981020.0,H,J,NaN,NaN,3,10200.0,1.0,33.051800,-115.581800,X,20940.0,2.0,0.0,6025.0


In [79]:
#placing function here to run storecount below on grocery_naics


def parent_name(row):
    '''Creates a new column called "PARENT NAME" based on the cleaned string match list''' 
    '''The new column generalizes individual companies into their parent coporation (ie. Walmart, Krogers, etc.)''' 
    companies = row['COMPANY']  
    
    if companies in walmart_clean:
        return 'WALMART'
    if companies in costco_clean:
        return 'COSTCO'
    if companies in ahold_clean:
        return 'AHOLD DELHAIZE'
    if companies in kroger_clean:
        return 'KROGER'
    if companies in albertsons_clean:
        return 'ALBERTSONS'
    else:
        return 'OTHER'

stores_2022['PARENT NAME'] = stores_2022.apply(parent_name, axis=1)

In [81]:
#filter by NAICS code "44511" - SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS

grocery_naics_2022 = stores_2022.loc[stores_2022[['PRIMARY NAICS CODE']].astype(str).apply(lambda x: x.str.startswith('44511')).any(axis=1)]
grocery_naics_2022

# There are 76579 supermarkets/related companies in the entire US in 2020

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE,PARENT NAME
2,EL SOL MARKET,110 W MAIN ST,WESTMORLAND,CA,92281,NaN,25.0,760,2,B,C,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,39106.0,5.0,1061.0,9,NaN,NaN,NaN,NaN,519605455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,10200.0,1.0,33.051800,-115.581800,X,20940.0,2.0,0.0,6025.0,OTHER
3,COOL SPRINGS GROCERY,241 WILDWOOD ST,MORGANTOWN,WV,26505,3141.0,61.0,304,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,39106.0,2.0,382.0,9,NaN,NaN,NaN,NaN,817953730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,10400.0,4.0,39.652885,-79.986398,P,34060.0,2.0,390.0,54061.0,OTHER
5,EL SOL MEAT MARKET,1100 MEADOWS RD # B,CALEXICO,CA,92231,5917.0,25.0,760,2,D,E,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,542107.0,MEAT-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,29.0,6153.0,9,NaN,NaN,NaN,NaN,995912391,NaN,NaN,NaN,NaN,NaN,NaN,995912391.0,NaN,7,12002.0,2.0,32.679923,-115.482166,P,20940.0,2.0,0.0,6025.0,OTHER
7,KROGER,1600 S OHIO ST,MARTINSVILLE,IN,46151,3317.0,109.0,765,2,F,G,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,39106.0,102.0,21931.0,2,P,NaN,NaN,NaN,140765967,NaN,7521503.0,1200.0,137888000.0,I,I,140765967.0,NaN,7,510701.0,2.0,39.411880,-86.425148,P,26900.0,2.0,294.0,18109.0,KROGER
8,FOOD 4 LESS,2420 COTTONWOOD DR,EL CENTRO,CA,92243,1604.0,25.0,760,2,E,F,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,39106.0,70.0,14851.0,2,ï¿½,NaN,NaN,NaN,215261702,402627138.0,7521503.0,1200.0,137888000.0,I,I,215261702.0,NaN,7,11201.0,1.0,32.815719,-115.572166,P,20940.0,2.0,0.0,6025.0,KROGER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203493,SAFEWAY,2204 W NOB HILL BLVD # A,YAKIMA,WA,98902,6200.0,77.0,509,2,F,G,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,546102.0,BAKERS-RETAIL,599201.0,FLORISTS-RETAIL,592102.0,LIQUORS-RETAIL,NaN,NaN,NaN,NaN,2022,NaN,145.0,35556.0,2,V,NaN,NaN,NaN,840802979,9606492.0,5995907.0,800.0,NaN,H,NaN,840802979.0,NaN,7,1100.0,1.0,46.584035,-120.539230,P,49420.0,2.0,0.0,53077.0,ALBERTSONS
203496,CORNUCOPIA,1104 THORPE LN # J,SAN MARCOS,TX,78666,7126.0,209.0,512,2,B,C,541108,GROCERS-HEALTH FOODS,44511007.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,549904.0,VITAMIN & FOOD SUPPLEMENTS,549913.0,HERBS,514949.0,NATURAL FOODS-WHOLESALE,549935.0,ORGANIC FOODS & SERVICES,512205.0,VITAMINS & FOOD SUPPLEMENTS-WHOLESALE,2022,NaN,6.0,1823.0,9,NaN,1970.0,NaN,NaN,164470890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,10304.0,1.0,29.886013,-97.924983,P,12420.0,2.0,0.0,48209.0,OTHER
203497,T-MART,103 SUNRISE DR,SUNRISE BEACH,TX,78643,9287.0,299.0,325,2,A,A,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,343201.0,PL

In [82]:
grocery_type= grocery_naics_2022["PRIMARY NAICS CODE"].unique().tolist()
grocery_type

[44511003.0, 44511006.0, 44511007.0, 44511004.0]

In [83]:
store_count_2022= grocery_naics_2022.groupby('PARENT NAME').count()[["CITY"]].reset_index().rename(columns={"CITY":"STORE COUNT"})
store_count_2022

,PARENT NAME,STORE COUNT
0,AHOLD DELHAIZE,1966
1,ALBERTSONS,2259
2,COSTCO,29
3,KROGER,2738
4,OTHER,68035
5,WALMART,16


## LET'S EXPLORE SOME VISUALIZATIONS 